In [1]:
import pandas as pd
import tqdm
import glob
import os

In [3]:
import datasets

In [5]:
ds = datasets.load_from_disk("../data/wiki270k")

In [19]:
df = ds.to_pandas()

In [21]:
df_wiki = df

In [27]:
df_wiki

,title,section,text
0,Russell Epstein,Russell Epstein,Russell Epstein is a professor of psychology a...
1,Russell Epstein,Education,Epstein received an undergraduate degree in ph...
2,Ambient device,Ambient device,Ambient devices are a type of consumer electro...
3,Ambient device,Purpose,The purpose of ambient devices is to enable im...
4,Ambient device,History,The concept of ambient devices can be traced b...
...,...,...,...
2101274,Symmetry,In the arts,In other arts and crafts Symmetries appear in ...
2101275,Symmetry,In the arts,In music Symmetry is not restricted to the vis...
2101276,Symmetry,In the arts,Pitch structures Symmetry is also an important...
2101277,Symmetry,In the arts,Interval cycles are symmetrical and thus non-d...


In [39]:
chunk_size = 3_000_000

In [42]:
def output_seperated_files(df_wiki, token_length, stride, drop_category):    
    output_dir = f"../data/wikipedia_fixed/sep_chunk_wiki270k/token_length{token_length}_stride_sentence{stride}_drop_category{drop_category}"
    os.makedirs(output_dir, exist_ok=True)
    chunk = 1
    df_wiki_sep = []
    for i in tqdm.tqdm(range(len(df_wiki))):
        series = df_wiki.iloc[i]
        text = series["text"].replace("\xa0", "")
        if drop_category:
            idx = text.find(" Category:")
            text = text[:idx]

        text_space = text.split(".")
        text = ""
        text_stride = []

        series_ = series.to_dict()
        series_["text"] = ".".join(text_space[:stride*2])
        series_["id"] = i
        series_["title_only"] = True
        df_wiki_sep.append(series_)

        appended = False
        for j in range(0, len(text_space)):
            series_ = series.to_dict()
            text += text_space[j] + "."
            text_stride.append(text_space[j])
            text_stride = text_stride[-stride:]
            appended = False
            if len(text.split(" ")) > token_length:
                series_["text"] = text
                series_["title_only"] = False
                df_wiki_sep.append(series_)
                text = ".".join(text_stride)
                appended = True
        if not appended:
            series_["text"] = text
            series_["title_only"] = False
            df_wiki_sep.append(series_)

        if len(df_wiki_sep) > chunk_size:
            print(f"chunk {chunk}")
            df_wiki_sep = pd.DataFrame(df_wiki_sep)
            df_wiki_sep.to_parquet(f"{output_dir}/chunk{chunk}.parquet")
            df_wiki_sep = []
            chunk += 1
    df_wiki_sep = pd.DataFrame(df_wiki_sep)
    df_wiki_sep.to_parquet(f"{output_dir}/chunk{chunk}.parquet")


In [43]:
output_seperated_files(df_wiki=df_wiki, token_length=120, stride=4, drop_category=True)

 56%|████████████████████████████████████▎                            | 1173801/2101279 [01:48<01:25, 10905.18it/s]

chunk 1


100%|█████████████████████████████████████████████████████████████████| 2101279/2101279 [03:25<00:00, 10221.16it/s]


In [44]:
df_a = pd.read_parquet( f"../data/wikipedia_fixed/sep_chunk_wiki270k/token_length120_stride_sentence4_drop_categoryTrue/chunk1.parquet")
df_a["title"] = df_a["title"].fillna("NaN.")
df_a.to_parquet(f"../data/wikipedia_fixed/sep_chunk_wiki270k/token_length120_stride_sentence4_drop_categoryTrue/chunk1.parquet")